In [178]:
import random
from together import Together
import os
import json

In [179]:
def txtToList(file_path):
    word_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            word = line.strip()
            if word:
                word_list.append(word)
    return word_list

In [180]:
nouns_path = 'words/nouns.txt'
verbs_path = 'words/verbs.txt'

nouns = txtToList(nouns_path)
verbs = txtToList(verbs_path)

In [181]:
action_types = {
    'Agents': 'Agents - Human, Living, Animate agents doing actions',
    'Enviroment': 'Environment - Just physics unfolding with objects',
    'Mixed': 'Mixed'
    }

time_scales = ['centuries','lot of years', 'a few years', 'a few months', 'lot of days', 'a few days', 'a few hours', 'a few minutes']

def gen_setting():
    n_nouns = random.randint(2, 5)
    n_verbs = random.randint(1, n_nouns - 1)
    n_para = random.randint(1,4)
    action_type = random.choice(list(action_types.keys()))
    time_scale = random.choice(time_scales)

    gen_nouns = [nouns[random.randint(0,len(nouns)-1)] for i in range(n_nouns)]
    gen_verbs = [verbs[random.randint(0,len(verbs)-1)] for i in range(n_verbs)]

    return {
        'n_para': n_para,
        'gen_nouns': gen_nouns,
        'gen_verbs': gen_verbs,
        'time_scale': time_scale,
        'action_type': action_type
    }

def build_prompt(s):
    return f"""
    You are a world sampler.
    Which captures a sequence of events in a world simulation and write it in form of a simple story.

    You have to frame it around the below nouns and verbs.
    Nouns: {', '.join(s['gen_nouns'])} (Use all of them, but pick one of them as the main subject)
    Verbs: {', '.join(s['gen_verbs'])}

    Time scale of the story is {s['time_scale']}
    
    The story should be {s['n_para']} paragraph{'s' if s['n_para'] > 1 else ''} long.

    We variation in entities we focus on based on these kind {', '.join(action_types.values())}
    In current story you will use: {action_types[s['action_type']]}

    Rules:
    - Language should be kindergarten level.
    - Do not include imaginative, fantastical or out-of-world things that would hinder real world model. Do no anthropomorphize non-living objects.
    - Adhere to physics and limits of real world.
    - Do not format, Do not use markdown, Do not use emojis.
    - The time scale should be implicit, not explicit. Do not put it everywhere for the sake of it.
    - The end state of the story should be a function of its beginning state. The events should unfold with casuality.
    - Do not lose flow and continuity, it shouldn't feel like random things forced together. It should feel like a thread.
    - When time scale is hours or minutes, you can use dialogues (if something that can speak is involved), sound effects of the physics or objects, visual imagery / description, Other sensory information.  

    """

In [182]:
api_key = os.getenv('TOGETHER_API_KEY')
client = Together(api_key=api_key)

In [183]:
def generate_story():
    settings = gen_setting()
    prompt = build_prompt(settings)
    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
            ]
    )

    story = response.choices[0].message.content
    return settings, story

In [185]:
_, story = generate_story()
print(sory)

NameError: name 'sory' is not defined

In [184]:
with open("dataset/tiny_reality_v1.jsonl", "a") as f:

    for i in range(100):
        data = generate_story()
        story_obj = {
            "settings": data[0],
            "story": data[1]
        }

        f.write(json.dumps(story_obj) + "\n")
        print(i)

f.close()

0
1
2
3
4
5
6
7
8
9
10


KeyboardInterrupt: 